In [1]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
from pymongo import MongoClient
from pandas import json_normalize
from pymongo import MongoClient,GEOSPHERE
import shapely.geometry
import geopandas

Ya tenemos lo que queremos priorizar, lo cual es, en orden de mas prioritario a menos
- Big starup
- Vegan
- Starbucks
- Vet
- Bar

In [2]:
load_dotenv()

True

In [3]:
client_id = os.getenv('Client_Id')
client_secret = os.getenv('Client_Secret')

In [4]:
url_query = 'https://api.foursquare.com/v2/venues/search'

In [5]:
def getFromDict(diccionario, mapa):
    from functools import reduce
    import operator
    return reduce(operator.getitem,mapa,diccionario)

In [6]:
def getAPI(url_query, coordenadas, query): #coordenadas en string, latitud y longitud (en ese orden)
    parametros = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f'{coordenadas}',
    "query": f"{query}",
    "limit": 200   
}
    resp = requests.get(url_query, params=parametros).json()['response']['venues']
    
    return resp

In [7]:
def limpiezaresp(json):
    
    mapa_nombre = ['name']
    mapa_latitud = ['location','lat']
    mapa_longitud = ['location', 'lng']
    mapa_ciudad = ['location','city']
    mapa_distancia = ['location','distance']
    
    jason = []
    
    for dic in json:
        try:
            diccio= {}
            diccio['nombre']= getFromDict(dic,mapa_nombre)
            diccio['latitud']= getFromDict(dic,mapa_latitud)
            diccio['longitud']= getFromDict(dic,mapa_longitud)
            diccio['ciudad']= getFromDict(dic,mapa_ciudad)
            diccio['distancia'] = getFromDict(dic,mapa_distancia)
            jason.append(diccio)
        except:
            pass
    return pd.DataFrame(jason)

In [8]:
def todo(url_query, coordenadas, query):
    api = getAPI(url_query, coordenadas, query)
    return limpiezaresp(api)

In [9]:
df_vegans_ny = todo('https://api.foursquare.com/v2/venues/search','40.75673, -73.98996', 'Vegan')

In [10]:
df_vegans_ny.head()

,nombre,latitud,longitud,ciudad,distancia
0,Omv! Vegan Burgers & More,40.757819,-73.985537,New York,392
1,Marty's Vegan Fast Foods,40.750163,-73.988871,New York,736
2,MTHR Vegan,40.762373,-73.987867,New York,652
3,Happy Zoe Vegan Bakery,40.719246,-73.944252,New York,5681
4,Maoz Vegan,40.751706,-73.974375,New York,1428


In [11]:
lista = ['Vegan', 'Start Up', 'Veterinary', 'Bar', 'Starbucks']

In [12]:
for i in lista:
    df_ny = todo('https://api.foursquare.com/v2/venues/search','40.75673, -73.98996', i)

In [13]:
df_full_ny = []
for i in lista:
    df_ny = todo('https://api.foursquare.com/v2/venues/search','40.75673, -73.98996', i)
    df_full_ny.append(df_ny)
df_NewYork = pd.concat(df_full_ny)
len(df_NewYork)

249

In [14]:
df_NewYork.head() # aqui tengo tooodo lo de Nueva York (Veganos, StartUps, Starbucks, Vets y Bares)

,nombre,latitud,longitud,ciudad,distancia
0,Omv! Vegan Burgers & More,40.757819,-73.985537,New York,392
1,Marty's Vegan Fast Foods,40.750163,-73.988871,New York,736
2,MTHR Vegan,40.762373,-73.987867,New York,652
3,Happy Zoe Vegan Bakery,40.719246,-73.944252,New York,5681
4,Maoz Vegan,40.751706,-73.974375,New York,1428


In [15]:
#Vamos a hacer lo mismo para Madrid 

In [16]:
df_full_mad = []
for i in lista:
    df_mad = todo('https://api.foursquare.com/v2/venues/search','40.50222, -3.67097', i)
    df_full_mad.append(df_mad)
df_Mad = pd.concat(df_full_mad)
df_Mad.head()

,nombre,latitud,longitud,ciudad,distancia
0,Vegan Streets Madrid Castellana,40.442456,-3.679973,Madrid,6696
1,Vegan Streets Madrid Centro,40.417512,-3.697636,Madrid,9696
2,Vegan bistro,40.419410,-3.679066,Madrid,9243
3,THUNDER VEGAN FOOD,40.424450,-3.702513,Madrid,9060
4,vegetarian / vegan restaurant,40.432974,-3.686693,Madrid,7822


In [17]:
#Ahora lo mismo para Barcelona

In [18]:
df_full_bcn = []
for i in lista:
    df_bcn = todo('https://api.foursquare.com/v2/venues/search','41.40080, 2.19763', i)
    df_full_bcn.append(df_bcn)
df_Bcn = pd.concat(df_full_bcn)
df_Bcn.head()

,nombre,latitud,longitud,ciudad,distancia
0,Vegan Streets Barcelona Glories,41.410040,2.193974,Barcelona,1072
1,Vegan Junk Food Bar,41.383386,2.180255,Barcelona,2421
2,Monchito Vegan,41.380971,2.179880,Barcelona,2658
3,ENJOY VEGAN,41.388152,2.180673,Barcelona,1996
4,"Good Shit, Vegan Kebabs",41.402854,2.156511,Barcelona,3441


Ya tenemos los tres dataframes que queremos.
Cada uno nos muestra todas nuestras 3 variables que tenemos en cuenta en esa ciudad, es decir, df_NewYork, me muestra todos los Veganos, todos los Starbucks, todos los Veterinarios, todos los Bares y todas las Start Ups.

DataFrames chulos:
- df_NewYork
- df_Mad
- df_Bcn

Ahora lo que tenemos que hacer es convertirlos en geodataframes y meterles una nueva columna que sean las coordenadas en tipo point para poder hacer las geoqueries de la manera correspondiente

In [19]:
# mira que guapa
# pero no la puedo usar, porque lo que viene despues, que son cosas de terminal que solo funcionan con mac...
'''
def export_to_Json (name, df):
    with open(f'{name}', 'w') as f:
        json.dump(df, f)
'''

"\ndef export_to_Json (name, df):\n    with open(f'{name}', 'w') as f:\n        json.dump(df, f)\n"

In [20]:
gdf_bcn = geopandas.GeoDataFrame(df_Bcn, geometry=geopandas.points_from_xy(df_Bcn.longitud,
                             df_Bcn.latitud))
gdf_bcn.columns=['nombre','lat','lng','ciudad','distancia','loc']
gdf_bcn.head()

,nombre,lat,lng,ciudad,distancia,loc
0,Vegan Streets Barcelona Glories,41.410040,2.193974,Barcelona,1072,POINT (2.19397 41.41004)
1,Vegan Junk Food Bar,41.383386,2.180255,Barcelona,2421,POINT (2.18026 41.38339)
2,Monchito Vegan,41.380971,2.179880,Barcelona,2658,POINT (2.17988 41.38097)
3,ENJOY VEGAN,41.388152,2.180673,Barcelona,1996,POINT (2.18067 41.38815)
4,"Good Shit, Vegan Kebabs",41.402854,2.156511,Barcelona,3441,POINT (2.15651 41.40285)


In [21]:
gdf_ny = geopandas.GeoDataFrame(df_NewYork, geometry=geopandas.points_from_xy(df_NewYork.longitud,
                             df_NewYork.latitud))
gdf_ny.columns=['nombre','lat','lng','ciudad','distancia','loc']
gdf_ny.head()

,nombre,lat,lng,ciudad,distancia,loc
0,Omv! Vegan Burgers & More,40.757819,-73.985537,New York,392,POINT (-73.98554 40.75782)
1,Marty's Vegan Fast Foods,40.750163,-73.988871,New York,736,POINT (-73.98887 40.75016)
2,MTHR Vegan,40.762373,-73.987867,New York,652,POINT (-73.98787 40.76237)
3,Happy Zoe Vegan Bakery,40.719246,-73.944252,New York,5681,POINT (-73.94425 40.71925)
4,Maoz Vegan,40.751706,-73.974375,New York,1428,POINT (-73.97437 40.75171)


In [22]:
gdf_mad = geopandas.GeoDataFrame(df_Mad, geometry=geopandas.points_from_xy(df_Mad.longitud,
                             df_Mad.latitud))
gdf_mad.columns=['nombre','lat','lng','ciudad','distancia','loc']
gdf_mad.head()

,nombre,lat,lng,ciudad,distancia,loc
0,Vegan Streets Madrid Castellana,40.442456,-3.679973,Madrid,6696,POINT (-3.67997 40.44246)
1,Vegan Streets Madrid Centro,40.417512,-3.697636,Madrid,9696,POINT (-3.69764 40.41751)
2,Vegan bistro,40.419410,-3.679066,Madrid,9243,POINT (-3.67907 40.41941)
3,THUNDER VEGAN FOOD,40.424450,-3.702513,Madrid,9060,POINT (-3.70251 40.42445)
4,vegetarian / vegan restaurant,40.432974,-3.686693,Madrid,7822,POINT (-3.68669 40.43297)


Nos exportamos estos 3 dataframes maravillosos en formato .csv para cargarlos en Mongo

In [23]:
gdf_ny.to_csv('newyork_data.csv')
gdf_mad.to_csv('mad_data.csv')
gdf_bcn.to_csv('bcn_data.csv')

Me hago una función que me convierta las coordenadas que le meto como string a tipo point para poder hacer las geoqueries debidamente en Mongo.
Ahora las coordenadas las pasamos en formato lista

In [24]:
coord_ny = [-73.98996, 40.75673]
coord_mad = [2.19763, 41.40080]
coord_bcn = [-3.67097, 40.50222]

In [25]:
def type_point(lista):
    return {"type": "Point", 
            "coordinates": lista
    }

In [26]:
type_point(coord_ny) # comprobandooo

{'type': 'Point', 'coordinates': [-73.98996, 40.75673]}

In [27]:
proj = {'id':0, 'name':1} # usaré esta proyección para que solo me de en nombre

In [30]:
client = MongoClient("localhost:27017")
db = client.get_database("GeoProject")

In [31]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'GeoProject')

Creamos los index de la geometry para cada geodataframe

In [32]:
db.NY.create_index([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [33]:
db.MAD.create_index([('geometry', GEOSPHERE)])

'geometry_2dsphere'

In [34]:
db.BCN.create_index([("geometry", GEOSPHERE)])

'geometry_2dsphere'

Genero una conexión con la base de datos

In [35]:
m = db.get_collection("MAD")
b = db.get_collection("BCN")
ny = db.get_collection("NY")